## Problem 2


experiment with a recently released Python implementation to compute Functional Maps on 3D meshes.
Explore this package and try to understand how to use it to map functions between two meshes and extract corresponding points between them.
Plot your results using your favorite visualization tools.

In [1]:
import numpy as np
from pyFM.pyFM.mesh import TriMesh
import meshplot as mp

In [2]:
def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)

def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

In [ ]:
import os

mesh_data = {}
for filename in os.listdir('./mesh_data'):
    if filename.endswith('.off'):
        mesh_data[filename[:-4]] = TriMesh(f"./mesh_data/{filename}", area_normalize=True, center=True).process(k=50, intrinsic=True)

for mesh in mesh_data.values():
    mesh_geod = mesh.get_geodesic(verbose=True)
    plot_mesh(mesh, mesh_geod)

  0%|          | 0/8595 [00:00<?, ?it/s]

Invalid color array given! Supported are numpy arrays. <class 'numpy.ndarray'>


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-6.005167…

  0%|          | 0/7207 [00:00<?, ?it/s]

Invalid color array given! Supported are numpy arrays. <class 'numpy.ndarray'>


...

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0408380…

  0%|          | 0/1002 [00:00<?, ?it/s]

Invalid color array given! Supported are numpy arrays. <class 'numpy.ndarray'>


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0409012…

  0%|          | 0/24003 [00:00<?, ?it/s]

# Computing Initial Correspondences
In this package, *functional map* usually go from Shape $1$ to Shape $2$, and *pointwise maps* from shape $2$ to shape $1$.

Let's compute the initial maps from `mesh2` to `mesh1`

In [3]:
print(mesh_data.keys())

dict_keys(['double_torus2', 'cat-00', 'torus_simple', 'lion-00', 'bunny_simple', 'bunny_simple2', 'torus', 'bunny'])


In [4]:
pairs_to_meach = [
    ('double_torus2', 'torus_simple'),
    ('torus', 'torus_simple'),
    ('bunny', 'bunny_simple'),
    ('bunny', 'bunny_simple2'),
    ('cat-00', 'lion-00'),
    ('cat-00', 'bunny'),
    ('lion-00', 'bunny'),
    ('double_torus2', 'lion-00')
    ]

for pair in pairs_to_meach:
    double_plot(mesh_data[pair[0]], mesh_data[pair[1]])


In [5]:
from pyFM.pyFM.signatures.WKS_functions import mesh_WKS
from pyFM.pyFM.spectral.nn_utils import knn_query

# using NN Initialisation mapping from mesh 2 to mesh 1
p2p_21_nn = []
for pair in pairs_to_meach:
    p2p_21_nn.append(knn_query(mesh_data[pair[0]].vertices, mesh_data[pair[1]].vertices, k=1))


mesh_WKS_dict = {mesh_name: mesh_WKS(mesh, num_E=100, k=100) for mesh_name, mesh in mesh_data.items()}

#using WKS initialisation mapping from mesh 1 to mesh 2
p2p_21_wks = []
for pair in pairs_to_meach:
    p2p_21_wks.append(knn_query(mesh_WKS_dict[pair[0]], mesh_WKS_dict[pair[1]], k=1))

In [6]:
# meching points from mesh 2 to mesh 1
for i, pair in enumerate(pairs_to_meach):
    cmap1 = visu(mesh_data[pair[0]].vertlist)
    cmap2_wks = cmap1[p2p_21_wks[i]]
    cmap2_nn = cmap1[p2p_21_nn[i]]

    double_plot(mesh_data[pair[0]],mesh_data[pair[1]],cmap1,cmap2_wks)
    double_plot(mesh_data[pair[0]],mesh_data[pair[1]],cmap1,cmap2_nn)


In [25]:
def plot_random_correspondence(mesh1, mesh2, p2p_map):
    import random

    # Choose a random vertex from mesh2
    i = random.randint(0, mesh2.vertlist.shape[0] - 1)
    j = p2p_map[i]  # p2p_map is (n,1) because k=1 in knn_query

    # Create a color map with all zeros and set the selected point to red
    color1 = np.zeros_like(mesh1.vertlist)
    color2 = np.zeros_like(mesh2.vertlist)
    color1[j] = np.array([1, 0, 0])  # Red
    color2[i] = np.array([1, 0, 0])  # Red

    # Plot the meshes with only the selected points highlighted
    double_plot(mesh1, mesh2, color1, color2)

In [26]:
# Visualize a random correspondence from WKS mapping
for i, pair in enumerate(pairs_to_meach):
    print(f"Random point match from {pair[1]} to {pair[0]}")
    plot_random_correspondence(mesh_data[pair[0]], mesh_data[pair[1]], p2p_21_wks[i])


Random point match from torus_simple to double_torus2


Random point match from torus_simple to torus


Random point match from bunny_simple to bunny


Random point match from bunny_simple2 to bunny


Random point match from lion-00 to cat-00


Random point match from bunny to cat-00


Random point match from bunny to lion-00


Random point match from lion-00 to double_torus2
